**Lab-08: Backpropagation**

Trong bài thực hành này chúng ta sẽ thử cài đặt Backpropagation 

Ta muốn dựa vào 2 chiều của lá, phân biệt giữa loại lá 1 và loại lá 2. Cụ thể, với $x = (x_1,x_2, 1)$ là input, ta muốn đoán một phân phối
    $$ P_\\theta(c|x),c = 0, 1 $$
với $\\theta$ là các tham số
Ta mô hình $P_\theta$ là một neural network có 2 lớp ẩn, mỗi lớp 5 neurons, tức là\n",
    $$ P_\\theta(c|x) = \\text{softmax}(\\max(0, \\max(0, x \\cdot W_1 + b_1) \\cdot W_2 + b_2) \\cdot W_3 + b_3 )$$

với $x$ là vector dòng $[[x_1, x_2]]$ kích thước $ 1\times 2$, $W_1, W_2, W_3$ là các ma trận có kích thước $2 \times 5, 5 \times 5, 5 \times 3$, và $b_1, b_2, b_3$ là các ma trận kích thước $1 \times 5, 1 \times 5, 1 \times 3$.

Khi đó $P(c|x)$ là một vector dòng độ dài 3, xem như $P(c|x)= (P_1(c|x), P_2(c|x), P_3(c|x)) = (P(c=0|x), P(c=1|x), P(c=2|x))$
Bộ các ma trận $\\theta = (W_1, W_2, W_3, b_1, b_2, b_3)$ chính là tham số cần tìm của model. Giờ cần tìm $\\theta$ sao cho 

$$ L = \frac{1}{N} \sum_{x,y} - y_0 \log P_\theta(0|x) -  y_1 \log P_\theta(1|x) - y_2 \log P_\theta(2|x) $$

đạt giá trị nhỏ nhất với $y = (y_0, y_1, y_2)$ là one-hot vector biểu thị loại lá tương ứng với $x$



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Preprocessing

In [ ]:
def one_hot_vector(y):
    out = np.zeros((y.shape[0], max(y)+1))
    for i in range(y.shape[0]):
        out[i, y[i]] = 1
    return out

In [ ]:
train = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-08/bt_train.csv")
valid = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/ML/master/lab-08/bt_valid.csv")

x1_train = train["x1"].values
x2_train = train["x2"].values
y_train = train["label"].values

x1_valid = valid['x1'].values
x2_valid = valid['x2'].values
y_valid = valid['label'].values

# normalize
x1_mean = np.mean(x1_train)
x1_std = np.std(x1_train)
x2_mean = np.mean(x2_train)
x2_std = np.std(x2_train)

x1_train = (x1_train - x1_mean)/ x1_std
x2_train = (x2_train - x2_mean)/ x2_std

x1_valid = (x1_valid - x1_mean)/ x1_std
x2_valid = (x2_valid - x2_mean)/ x2_std



X_train = np.concatenate([x1_train.reshape(-1,1), x2_train.reshape(-1,1)], axis=1)
y_train = one_hot_vector(y_train)

X_valid = np.concatenate([x1_valid.reshape(-1,1), x2_valid.reshape(-1,1)], axis=1)

In [ ]:
# initialize
W1 = np.random.randn(2,5)
W2 = np.random.randn(5,5)
W3 = np.random.randn(5,3)

b1 = np.random.randn(1,5)
b2 = np.random.randn(1,5)
b3 = np.random.randn(1,3)

In [ ]:
def relu(h):
    return np.array([max(0,i) for i in h.reshape(-1)]).reshape(h.shape)

def softmax(z):
    return np.exp(z)/ np.sum(np.exp(z), axis=1).reshape(-1,1)

def CrossEntropy(o,y):
    return - np.sum(np.log(o)*y)

ln = 0.001
N = y_train.shape[0]

In [ ]:
for epochs in range(100000):
    # foward
    z1 = np.dot(X_train, W1) + b1
    o1 = relu(z1)

    z2 = np.dot(o1, W2) + b2
    o2 = relu(z2)

    z3 = np.dot(o2, W3) + b3
    o3 = softmax(z3)

    # backpropagation
    dL_dz3 = 1/len(X_train)*(o3 - y_train) 
    dL_dW3 = np.dot(o2.T, dL_dz3)    
    dL_db3 = np.sum(dL_dz3, axis = 0)


    dL_do2 = np.dot(dL_dz3, W3.T)
    dL_dz2 = dL_do2.copy()
    dL_dz2[z2 < 0] = 0
    dL_dW2 = np.dot(o1.T,dL_dz2)
    dL_db2 = np.sum(dL_dz2, axis = 0)

    dL_do1 = np.dot(dL_dz2, W2.T)
    dL_dz1 = dL_do1.copy()
    dL_dz1[z1 < 0] = 0
    dL_dW1 = np.dot(X_train.T, dL_dz1)
    dL_db1 = np.sum(dL_dz1, axis = 0)

    W3 -= ln* dL_dW3
    b3 -= ln* dL_db3
    W2 -= ln* dL_dW2
    b2 -= ln* dL_db2
    W1 -= ln* dL_dW1
    b1 -= ln* dL_db1


In [ ]:
z1_valid = np.dot(X_valid, W1) + b1
o1_valid = relu(z1_valid)

z2_valid = np.dot(o1_valid, W2) + b2
o2_valid = relu(z2_valid)

z3_valid = np.dot(o2_valid, W3) + b3
o3_valid = softmax(z3_valid)

In [ ]:
np.sum(np.argmax(o3_valid, axis = 1) == y_valid)/ y_valid.shape[0]

0.6233333333333333

#Bài Tập
1. Từ code demo hãy cài đặt thêm một module để chọn ra được bộ weights sao cho accuracy trên tập validation là tốt nhất.
2. Từ bộ dữ liệu bên dưới hãy cài đặt backpropagation cho bài toán phân biệt ung thư vú. Hãy tự chọn số layers và số nodes mà mình cho là thích hợp, cũng như là nêu ra số layers và số nodes của mỗi layer mà mình đã chọn. Tính accuracy trên tập training.

In [ ]:
from sklearn import datasets

breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data  
y = breast_cancer.target

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split( X, y, test_size=0.2, random_state=42)

X_mean=np.mean(X_train)
X_std=np.std(X_train)

X_valid=(X_valid-X_mean)/X_std
    X_train=(X_train-X_mean)/X_std